Preparing necessary imports

In [9]:
from ipywidgets import widgets, interact, fixed
from IPython.display import display
from getpass import getpass
import omero.gateway as gw
import omero
import pandas as pd

from plots import plot_homogeneity_map, plot_distances_map, plot_psfs, plot_distances_map_bokeh
from utils import get_microscope_history

from bokeh.io import output_notebook, reset_output, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import ColorBar, LinearColorMapper, LogColorMapper, Label

reset_output()
output_notebook()  # show output in notebook

try:
    from credentials import *
except ModuleNotFoundError:
    HOST = input('host:')
    PORT = 4064
    USER = input('user:')
    PASSWORD = getpass('password')
    GROUP = input('group:')

Loading BokehJS ...

### Open connection

In [10]:
conn = gw.BlitzGateway(username=USER,
                       passwd=PASSWORD,
                       host=HOST,
                       group=GROUP,
                       port=PORT)

# conn = gw.BlitzGateway(username=input('OMERO username:'), password=getpass('OMERO password:'),
#                        group=GROUP, host=OMERO_SERVER, port=PORT, secure=True)

conn.connect()
print(f'Connection successful: {conn.isConnected()}')

Connection successful: True


### Selecting the Microscope

In [11]:
microscopes = list(conn.getObjects('Project'))
micr_menu = widgets.Dropdown(options=[(m.getName(), i) for i, m in enumerate(microscopes)],
                             value=1,
                             description='Select a microscope:',
                             disabled=False,)
display(micr_menu)

Dropdown(description='Select a microscope:', index=1, options=(('microscope_2', 0), ('microscope_1', 1)), valu…

In [12]:
microscope = microscopes[micr_menu.value]
history_df = get_microscope_history(microscope)
history_df

,Acquisition_date_time,Analysis_date_time,Distance_units,FWHM_units,Max_Intensity_Roi_ch00,Max_Intensity_Roi_ch01,Max_Intensity_Roi_ch02,Max_Intensity_Roi_ch03,Max_Intensity_ch00,Max_Intensity_ch01,...,measured_band,resolution_axis,resolution_units,405,488,532,561,642,date,power_units
0,2017-02-20,2020-04-29 15:46:39.780914,MICROMETER,MICROMETER,7,15,22,14,7689053.0,6569755.0,...,0.2,1,MICROMETER,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-03-20,2020-05-05 02:29:15.333720,MICROMETER,NaN,21,2,14,1,8162913.0,7016200.0,...,0.2,1,MICROMETER,,,,,,2020-05-05,mW
2,2017-04-20,2020-05-05 02:27:07.521182,MICROMETER,NaN,18,5,26,20,7997067.0,6862454.0,...,0.2,1,MICROMETER,,,,,,2020-05-05,mW
3,2017-05-20,2020-05-05 02:25:07.871273,MICROMETER,NaN,9,2,23,18,7806755.0,6663374.0,...,0.2,1,MICROMETER,,,,,,2020-05-05,mW
4,2017-06-20,2020-05-05 02:23:01.372189,MICROMETER,NaN,20,8,29,21,8078817.999999999,6953084.0,...,0.2,1,MICROMETER,,,,,,2020-05-05,mW
5,2017-07-20,2020-05-05 02:20:50.121894,MICROMETER,NaN,20,6,29,21,8033405.0,6911205.0,...,0.2,1,MICROMETER,,,,,,2020-05-05,mW
6,2017-08-20,2020-05-05 02:18:49.896513,MICROMETER,NaN,9,2,23,18,7796846.0,6653965.0,...,0.2,1,MICROMETER,,,,,,2020-05-05,mW
7,2017-09-20,2020-05-05 02:16:48.090351,MICROMETER,NaN,11,3,23,19,7836707.0,6696244.0,...,0.2,1,MICROMETER,,,,,,2020-05-05,mW
8,2017-10-20,2020-05-05 02:14:47.683512,MICROMETER,NaN,17,3,24,17,7911267.0,6797738.0,...,0.2,1,MICROMETER,,,,,,2020-05-05,mW
9,2017-11-20,2020-05-05 02:12:46.988938,MICROMETER,NaN,7,15,22,14,7689053.0,6569755.0,...,0.2,1,MICROMETER,,,,,,2020-05-05,mW


In [13]:
variable_menu = widgets.Dropdown(options=list(history_df.columns),
                                 description='Select a variable to display',
                                 disabled=False)
display(variable_menu)

Dropdown(description='Select a variable to display', options=('Acquisition_date_time', 'Analysis_date_time', '…

In [26]:
history_df['Acquisition_date_time'] = pd.to_datetime(history_df['Acquisition_date_time'])

p = figure(plot_width=800, plot_height=250, x_axis_type="datetime")

p.line(history_df['Acquisition_date_time'], history_df[variable_menu.value], color='navy', alpha=0.5)

show(p)